# Module 5. Classification part 1

# Data preparation

## Lecture objectives

1. Introduce the principles of machine learning
2. Provide more practice with data wrangling

Machine learning is a very general term that covers many parts of data science. Here, we will look at two specific problems that machine learning is well equipped to handle:
* Classification (this module and the next)
* Clustering (the subsequent module)

As a broad generalization, machine learning-based classification focuses on *prediction*. For example: [which neighborhoods are likely to gentrify](https://journals.sagepub.com/doi/abs/10.1177/0042098018789054)? [Which facilities are likely to be violating environmental standards?](https://www.nature.com/articles/s41893-018-0142-9) What is demand likely to be at a new bikeshare station? [What is the race and gender of an author on a course reading list](http://syllabusdiversity.org)?

There are also applications which raise more concerns with ethics and justice (yes, [predictive policing](https://www.technologyreview.com/2020/07/17/1005396/predictive-policing-algorithms-racist-dismantled-machine-learning-bias-criminal-justice/), I'm talking about you). We'll come back to these issues in a couple of weeks.

Machine learning is less successful with questions of *causation* and *hypothesis testing*. Here, a statistical approach (frequentist or Bayesian) is likely to be more appropriate, although there is quite a bit of overlap between "statistics" and "machine learning."

There are at least three widely used approaches to classification.
* Logistic regression. This is often used in a more statistical setting, but is the starting point for much machine learning analysis. 
* Random forests. We'll focus on this technique.
* Neural networks. Often used for image recognition, this can be a "black box" approach to prediction and classification.

Important: machine learning is a very large field, and there are entire courses on the theory and applications. Here, we will give a very high-level overview. We'll focus on the big-picture applicability of machine learning techniques, and actually implementing them in Python. We'll skate over the theoretical underpinnings and the details of the various algorithms.

## Example: ADUs in LA
The example we will use is whether property owners construct Accessory Dwelling Units (ADUs) in the City of Los Angeles. You might imagine that a predictive approach could be useful to planners and policymakers. Not least, they could predict future ADU growth, and the neighborhoods where ADUs are most likely to be built.

We can obtain the data from the City's building permits database (which tells us whether or not an ADU was built), and the County Assessor parcel database (which provides covariates such as lot size). Because both of these datasets are very large, I preprocessed them and saved a slimmed-down version that is in your GitHub folder. Specifically, I extracted a subset of fields, limited the building permits to those that include an ADU, and limited the parcels to those in the City of LA.

## Wrangling the data
We have two input data files: permits and parcels. The aim: add a column to the parcels dataframe that is `True` if an ADU has been permitted on that parcel, and `False` otherwise.
    
Even with this preprocessing, there is some work to do in joining the datasets together. 

In [1]:
import pandas as pd

# get building permit data
# this is an abbreviated version of the data here (>500 MB):
# https://data.lacity.org/City-Infrastructure-Service-Requests/Building-and-Safety-Permit-Information-Old/yv23-pmwf

# this code was used to read in the data and save a subset (ADU permits only)
# that is manageable in size
if 0:  # if 0 means this block won't be executed (because 0 is False)
    cols_to_use = ['Assessor Book', 'Assessor Page', 'Assessor Parcel', '# of Accessory Dwelling Units']
    df = pd.read_csv('Building_and_Safety_Permit_Information_Old.csv', usecols=cols_to_use)
    df = df[df['# of Accessory Dwelling Units']>0]
    df.to_csv('ADU_permits.csv', index=False)

permits = pd.read_csv('../data/ADUs/ADU_permits.csv')  # this file should be in your GitHub folder
permits.head()    

,Assessor Book,Assessor Page,Assessor Parcel,# of Accessory Dwelling Units
0,2126.0,24.0,041,1.0
1,2245.0,10.0,006,1.0
2,2640.0,7.0,014,1.0
3,2671.0,22.0,024,1.0
4,2301.0,9.0,003,1.0


In [3]:
# original data: https://egis-lacounty.hub.arcgis.com/datasets/parcels
# this code was used to read in the data and save a subset 
# (City of LA only, subset of columns) that is manageable in size

import geopandas as gpd

if 0: # if 0 means this block won't be executed
    gdf = gpd.read_file('LACounty_Parcels.gdb', driver='FileGDB', layer='LACounty_Parcels')
    gdf.dropna(subset=['SitusCity'], inplace=True)
    gdf = gdf[gdf['SitusCity'].str.startswith('LOS ANGELE')]
    cols_to_use = ['APN', 'UseType', 'UseDescription','YearBuilt1', 'Units1','Bedrooms1', 'Bathrooms1', 
         'SQFTmain1','Roll_LandValue', 'Roll_ImpValue', 'Roll_LandBaseYear', 'Roll_ImpBaseYear', 'CENTER_LAT', 'CENTER_LON']
    parceldf = pd.DataFrame(gdf)[cols_to_use]  # drops the geometry column as well
    parceldf.to_csv('parcels.csv', index=False)
    del gdf   # frees up space

parcels = pd.read_csv('../data/ADUs/parcels.csv')
parcels.head()

,APN,UseType,UseDescription,YearBuilt1,Units1,Bedrooms1,Bathrooms1,SQFTmain1,Roll_LandValue,Roll_ImpValue,Roll_LandBaseYear,Roll_ImpBaseYear,CENTER_LAT,CENTER_LON
0,8543-009-073,Residential,Single,1940.0,1.0,3.0,2.0,1174.0,111510.0,112960.0,2000,2000,34.093940,-117.968917
1,8028-013-011,Residential,Two Units,1901.0,1.0,2.0,1.0,1115.0,117403.0,189350.0,1988,1988,33.921639,-118.043151
2,7563-037-055,Residential,Single,1962.0,1.0,4.0,2.0,2153.0,45925.0,89381.0,1975,1975,33.720274,-118.325022
3,7563-037-054,Residential,Single,1959.0,1.0,3.0,2.0,1862.0,676304.0,169069.0,2008,2008,33.720115,-118.325069
4,7563-037-053,Residential,Single,1965.0,1.0,3.0,2.0,2206.0,762619.0,190654.0,2020,2020,33.719947,-118.325116


Note that the `APN` column in `parcels` has a format that corresponds to three columns in `permits`: `Assessor Book`-`Assessor Page`-`Assessor Parcel`. 

So the first step is to create this `APN` column in `permits`.

We first convert each column to an integer (to drop the decimal point), then to a string, then pad with zeros, and then concatenate the columns separated by `-`.

In [4]:
# join
permits['APN'] = (permits['Assessor Book'].astype(int).astype(str).str.zfill(4) + '-' 
                   + permits['Assessor Page'].astype(int).astype(str).str.zfill(3) + '-'
                   + permits['Assessor Parcel'].astype(int).astype(str).str.zfill(3))

ValueError: invalid literal for int() with base 10: '***'

What happened? Note two things:
* The problem is that we are trying to convert `'***'` to an integer
* The error is being caused by the `permits['Assessor Parcel'].astype(int)` part of the code.

So let's look at those rows.

In [5]:
permits[permits['Assessor Parcel']=='***'].head()

,Assessor Book,Assessor Page,Assessor Parcel,# of Accessory Dwelling Units
2283,2307.0,23.0,***,1.0
3294,2248.0,2.0,***,1.0
4146,2373.0,11.0,***,1.0
6753,2219.0,6.0,***,1.0
9138,2653.0,23.0,***,6.0


It seems like the parcel number is just missing, so let's drop them.

Note the `!=` operator means "not equal to." So we are keeping the rows that are *not* `***`.

In [6]:
permits = permits[permits['Assessor Parcel']!='***']

Now let's try to create the column again.

In [7]:
permits['APN'] = (permits['Assessor Book'].astype(int).astype(str).str.zfill(4) + '-' 
                   + permits['Assessor Page'].astype(int).astype(str).str.zfill(3) + '-'
                   + permits['Assessor Parcel'].astype(int).astype(str).str.zfill(3))
permits.head()

,Assessor Book,Assessor Page,Assessor Parcel,# of Accessory Dwelling Units,APN
0,2126.0,24.0,041,1.0,2126-024-041
1,2245.0,10.0,006,1.0,2245-010-006
2,2640.0,7.0,014,1.0,2640-007-014
3,2671.0,22.0,024,1.0,2671-022-024
4,2301.0,9.0,003,1.0,2301-009-003


<div class="alert alert-block alert-info">
<strong>Question:</strong> What type of join do we want? Left? Right? Inner? Outer? 1:1? 1:many?
</div>

Note two things:
* We need to keep all of the parcels, even if there isn't a corresponding permit. Otherwise, we can't do any prediction—we'd have a dataset where *every* parcel has an ADU. So that implies a left join to the parcels dataframe
* We don't want to duplicate parcels. So let's drop any duplicates (on the APN column) in both the permit and parcels dataframes. That will guarantee a 1:1 join

Let's first check to see if duplicates exist.

In [8]:
permits.APN.is_unique

False

In [9]:
parcels.APN.is_unique

False

There are two ways to drop duplicates: the [pandas `drop_duplicates()` function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html) is one.

But sometimes it's easier to use `groupby`, and then take the first in each group. If there is only one row in a group, it will be returned unchanged.

A byproduct of using `groupby` on the `APN` column is that `APN` is now our index. That will make the join easier.

In [10]:
# in the permits, take the first row of any duplicates for convenience
print('Before dropping duplicates: {}'.format(len(permits)))
permits = permits.groupby('APN').first()
print('After dropping duplicates: {}'.format(len(permits)))
permits.index.is_unique  # make sure the index (APN) is unique

Before dropping duplicates: 26466
After dropping duplicates: 25535


True

In [11]:
permits.head()

,Assessor Book,Assessor Page,Assessor Parcel,# of Accessory Dwelling Units
APN,,,,
2004-004-010,2004.0,4.0,010,1.0
2004-004-011,2004.0,4.0,011,1.0
2004-004-013,2004.0,4.0,013,1.0
2004-004-019,2004.0,4.0,019,1.0
2004-006-001,2004.0,6.0,001,1.0


In [12]:
print('Before dropping duplicates: {}'.format(len(parcels)))
parcels = parcels.groupby('APN').first()
print('After dropping duplicates: {}'.format(len(parcels)))
parcels.index.is_unique  # make sure the index (APN) is unique

Before dropping duplicates: 790807
After dropping duplicates: 790805


True

In [13]:
parcels.head()

,UseType,UseDescription,YearBuilt1,Units1,Bedrooms1,Bathrooms1,SQFTmain1,Roll_LandValue,Roll_ImpValue,Roll_LandBaseYear,Roll_ImpBaseYear,CENTER_LAT,CENTER_LON
APN,,,,,,,,,,,,,
2004-001-003,Residential,Single,1973.0,1.0,4.0,3.0,2090.0,684054.0,289482.0,2006,2006,34.220220,-118.620669
2004-001-004,Residential,Single,1973.0,1.0,5.0,3.0,2479.0,356150.0,245944.0,2010,2010,34.220039,-118.620668
2004-001-005,Residential,Single,1973.0,1.0,4.0,2.0,2057.0,505922.0,190867.0,2018,2018,34.219858,-118.620676
2004-001-008,Residential,Single,1978.0,1.0,4.0,3.0,2423.0,123435.0,213347.0,1980,1980,34.220334,-118.622706
2004-001-009,Residential,Single,1978.0,1.0,4.0,3.0,2226.0,134500.0,201858.0,1984,1984,34.220323,-118.623050


<div class="alert alert-block alert-info">
<strong>Exercise:</strong> How would you join the permits dataframe to the parcels?
</div>

In [14]:
joinedDf = parcels.join(permits, how='left') # left is the default so we could omit that argument
print('N parcels: {}'.format(len(joinedDf)))
print('N joined: {}'.format(joinedDf['# of Accessory Dwelling Units'].count()))
joinedDf.head()

N parcels: 790805
N joined: 25161


,UseType,UseDescription,YearBuilt1,Units1,Bedrooms1,Bathrooms1,SQFTmain1,Roll_LandValue,Roll_ImpValue,Roll_LandBaseYear,Roll_ImpBaseYear,CENTER_LAT,CENTER_LON,Assessor Book,Assessor Page,Assessor Parcel,# of Accessory Dwelling Units
APN,,,,,,,,,,,,,,,,,
2004-001-003,Residential,Single,1973.0,1.0,4.0,3.0,2090.0,684054.0,289482.0,2006,2006,34.220220,-118.620669,NaN,NaN,NaN,NaN
2004-001-004,Residential,Single,1973.0,1.0,5.0,3.0,2479.0,356150.0,245944.0,2010,2010,34.220039,-118.620668,NaN,NaN,NaN,NaN
2004-001-005,Residential,Single,1973.0,1.0,4.0,2.0,2057.0,505922.0,190867.0,2018,2018,34.219858,-118.620676,NaN,NaN,NaN,NaN
2004-001-008,Residential,Single,1978.0,1.0,4.0,3.0,2423.0,123435.0,213347.0,1980,1980,34.220334,-118.622706,NaN,NaN,NaN,NaN
2004-001-009,Residential,Single,1978.0,1.0,4.0,3.0,2226.0,134500.0,201858.0,1984,1984,34.220323,-118.623050,NaN,NaN,NaN,NaN


That seems good enough. We join almost all of the permits to the parcels dataframe. 

Now let's create a column that is 0 if there is no ADU (i.e., if the permit data did not join), and 1 otherwise.

We'll use our `lambda` function again. If the value of the column is Null (using the handy `pd.isnull`), we'll return `False`. Otherwise, `True`.

In [15]:
joinedDf['hasADU'] = joinedDf['# of Accessory Dwelling Units'].apply(
                        lambda x: False if pd.isnull(x) else True)
joinedDf.head()

,UseType,UseDescription,YearBuilt1,Units1,Bedrooms1,Bathrooms1,SQFTmain1,Roll_LandValue,Roll_ImpValue,Roll_LandBaseYear,Roll_ImpBaseYear,CENTER_LAT,CENTER_LON,Assessor Book,Assessor Page,Assessor Parcel,# of Accessory Dwelling Units,hasADU
APN,,,,,,,,,,,,,,,,,,
2004-001-003,Residential,Single,1973.0,1.0,4.0,3.0,2090.0,684054.0,289482.0,2006,2006,34.220220,-118.620669,NaN,NaN,NaN,NaN,False
2004-001-004,Residential,Single,1973.0,1.0,5.0,3.0,2479.0,356150.0,245944.0,2010,2010,34.220039,-118.620668,NaN,NaN,NaN,NaN,False
2004-001-005,Residential,Single,1973.0,1.0,4.0,2.0,2057.0,505922.0,190867.0,2018,2018,34.219858,-118.620676,NaN,NaN,NaN,NaN,False
2004-001-008,Residential,Single,1978.0,1.0,4.0,3.0,2423.0,123435.0,213347.0,1980,1980,34.220334,-118.622706,NaN,NaN,NaN,NaN,False
2004-001-009,Residential,Single,1978.0,1.0,4.0,3.0,2226.0,134500.0,201858.0,1984,1984,34.220323,-118.623050,NaN,NaN,NaN,NaN,False


<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Rewrite the above line of code using a regular function rather than an anonymous (lambda) function. Verify you get the same result.
</div>

In [16]:
def hasADU(n_ADUs):
    if pd.isnull(n_ADUs):
        return False
    else:
        return True
joinedDf['hasADU2'] = joinedDf['# of Accessory Dwelling Units'].apply(hasADU)

joinedDf.hasADU2 == joinedDf.hasADU

APN
2004-001-003    True
2004-001-004    True
2004-001-005    True
2004-001-008    True
2004-001-009    True
                ... 
7563-037-053    True
7563-037-054    True
7563-037-055    True
8028-013-011    True
8543-009-073    True
Length: 790805, dtype: bool

In [17]:
# np.all verifies that all elements of an array are True
import numpy as np
print(np.all(joinedDf.hasADU2 == joinedDf.hasADU))

# delete the temporary column
joinedDf.drop(columns=['hasADU2'], inplace=True)

True


Let's stop there for now. We'll save the data so that we can reload it at the start of the next video lecture.

You could save it as a `csv`. But we can also save the pandas DataFrame object, through "pickling" it. This is convenient when you want to save something temporarily, but it's not advisable for long-term archiving or sharing your work.

In [18]:
joinedDf.to_pickle('../scratch/joined_permits.pandas')

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Machine learning is particularly valuable for prediction, and when there are many highly correlated variables.</li>
  <li>Data wrangling is almost always your first step, and joins will come with practice.</li>
</ul>
</div>